In [1]:
#r "nuget: Langchain, *-*"
#r "nuget: Langchain.Core,*-*"
#r "nuget: LangChain.DocumentLoaders.Pdf, *-*"
#r "nuget: LangChain.Databases.Sqlite, *-*"
#r "nuget: LangChain.Providers.Ollama, *-*"
#r "nuget: LangChain.Splitters.CSharp, *-*"

Installed Packages Langchain, 0.14.1-dev.8 Langchain.Core, 0.14.1-dev.8 LangChain.Databases.Sqlite, 0.14.1-dev.8 LangChain.DocumentLoaders.Pdf, 0.14.1-dev.8 LangChain.Providers.Ollama, 0.14.1-dev.8 LangChain.Splitters.CSharp, 0.14.1-dev.8

In [2]:
using LangChain.Base;
using LangChain.Databases.Sqlite;
using LangChain.DocumentLoaders;
using LangChain.Providers;
using static LangChain.Chains.Chain;
using LangChain.Providers.Ollama;
using LangChain.Splitters.Text;
using LangChain.Extensions;
using OpenAI.Constants;
using System.Net.NetworkInformation;
using DataSource = LangChain.DocumentLoaders.DataSource;
using System.Threading;

In [3]:

var embeddingModel = new OllamaEmbeddingModel(new OllamaProvider(), id: "all-minilm");
var llm = new OllamaLanguageModelInstruction(
    "llama3",
    options: new OllamaLanguageModelOptions
    {
        Stop = ["\n"],
        Temperature = 0.0f,
    });

var vectorDatabase = new SqLiteVectorDatabase(dataSource: "vectors.db");

In [4]:
var vectorCollection = await vectorDatabase.AddDocumentsFromAsync<PdfPigPdfLoader>(
    embeddingModel, // Used to convert text to embeddings
    dimensions: 1536, // Should be 1536 for TextEmbeddingV3SmallModel
    dataSource: DataSource.FromUrl("https://canonburyprimaryschool.co.uk/wp-content/uploads/2016/01/Joanne-K.-Rowling-Harry-Potter-Book-1-Harry-Potter-and-the-Philosophers-Stone-EnglishOnlineClub.com_.pdf"),
    collectionName: "harrypotter", // Can be omitted, use if you want to have multiple collections
    textSplitter: null);

In [5]:
const string question = "What is Harry's Address?";
var similarDocuments = await vectorCollection.GetSimilarDocuments(embeddingModel, question, amount: 5);
CancellationToken cancellationToken = default;
// Use similar documents and LLM to answer the question
var answer = await llm.GenerateAsync(
    $"""
     Use the following pieces of context to answer the question at the end.
     If the answer is not in context then just say that you don't know, don't try to make up an answer.
     Keep the answer as short as possible.

     {similarDocuments.AsString()}

     Question: {question}
     Helpful Answer:
     """, cancellationToken: cancellationToken).ConfigureAwait(false);

Console.WriteLine($"LLM answer: {answer}"); // The cloaked figure.

LLM answer:  The address written on the letter is: Mr H. Potter The Cupboard under the Stairs 4 Privet Drive Little Whinging Surrey
